In [1]:
import numpy as np
import socket
from PIL import Image
from keras.models import load_model

width = 64
height = 64
channel = 3

## Load the models

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Загружаем модель без компиляции
lidar_model = load_model('car_model_lidar.h5', compile=False)
camera_model = load_model('car_model_camera.h5', compile=False)

new_optimizer = Adam(learning_rate=7.5e-05)  # старое lr заменяем на learning_rate

# Повторно компилируем модели с новым оптимизатором
lidar_model.compile(optimizer=new_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
camera_model.compile(optimizer=new_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# lidar_model = load_model('car_model_lidar.h5')
# camera_model = load_model('car_model_camera.h5')

C:\Users\m.bergaliyev\AppData\Local\anaconda3\envs\tf-cpu\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\m.bergaliyev\AppData\Local\anaconda3\envs\tf-cpu\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [3]:
print(lidar_model.summary())

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_0 (Conv2D)               │ (None, 62, 62, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lrelu_0 (LeakyReLU)             │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_0 (MaxPooling2D)        │ (None, 20, 20, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 18, 18, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lrelu_1 (LeakyReLU)             │ (None, 18, 18, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling2D)        │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lrelu_2 (LeakyReLU)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling2D)        │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lrelu_4 (LeakyReLU)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_classes (Dense)          │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,939 (109.14 KB)

 Trainable params: 27,939 (109.14 KB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
print(camera_model.summary())

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_0 (Conv2D)   │ (None, 62, 62,    │        448 │ image_input[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lrelu_0 (LeakyReLU) │ (None, 62, 62,    │          0 │ conv2d_0[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_0           │ (None, 20, 20,    │          0 │ lrelu_0[0][0]     │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 18, 18,    │      4,640 │ maxpool_0[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lrelu_1 (LeakyReLU) │ (None, 18, 18,    │          0 │ conv2d_1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_1           │ (None, 6, 6, 32)  │          0 │ lrelu_1[0][0]     │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 4, 4, 64)  │     18,496 │ maxpool_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lrelu_2 (LeakyReLU) │ (None, 4, 4, 64)  │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_2           │ (None, 1, 1, 64)  │          0 │ lrelu_2[0][0]     │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 64)        │          0 │ maxpool_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lrelu_4 (LeakyReLU) │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_lane (Dense) │ (None, 2)         │        130 │ lrelu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_angle        │ (None, 1)         │         65 │ lrelu_4[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 27,939 (109.14 KB)

 Trainable params: 27,939 (109.14 KB)

 Non-trainable params: 0 (0.00 B)

None


## Class for environment & agent interactions

In [5]:
class EnvironmentInteraction:
    
    def __init__(self, Address, Port):
        
        # This byte array like delimeter uses to divide input byte socket stream to camera and lidar image
        self.byteDelimeter = b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4
        
        # The ip address of Unity client, usually set to null
        self.address = Address
        
        # The port to listen
        self.port = Port
        
        # The socket object
        self.sock = socket.socket()
        
        # Init socket object
        self.sock.bind((self.address, self.port))
            
        
    def listen(self):
        
        # Start to listen
        self.sock.listen(1)
        
        # Store connection when get request
        self.connection, self.address = self.sock.accept()
        
        
    def step(self, action):
        
        # Send the action to environment
        self.connection.send(action.encode('utf-8'))                        # Send action to environment
        
        # Receive the next images
        data = self.connection.recv(768432)                                 # Get camera and lidar images as byte array
        
        # Split to two byte arrays
        data = data.split(self.byteDelimeter)
        
        # Save the byte arrays as images
        camera = Image.frombytes('RGB', (64, 64), data[0])
        lidar = Image.frombytes('RGB', (64, 64), data[1])
        
        # Convert to numpy ndarray with 64x64x3 shape to flip images
        camera = np.array(camera.getdata(), dtype = np.float32).reshape(width, height, channel)
        lidar = np.array(lidar.getdata(), dtype = np.float32).reshape(width, height, channel)
        
        # Normalize the data
        camera = camera / 255
        lidar = lidar / 255
        
        # Flip the images horizontally
        camera = np.flip(camera, axis = 0)    
        lidar = np.flip(lidar, axis = 0)
        
        # Convert to numpy ndarray with 1x64x64x3 shape to predict by models
        camera = np.array(camera, dtype = np.float32).reshape(1, width, height, channel)
        lidar = np.array(lidar, dtype = np.float32).reshape(1, width, height, channel)
                
        return camera, lidar

# Object of EnvironmentInteraction class
environment = EnvironmentInteraction('', 7777) 

In [6]:
lidar_target_names = ['Human', 'Clear path', 'Obstacle']
lane_target_names = ['Left Lane', 'Right Lane']
counter = 0
obstacleDetected = False

# Function for formatting the command to environment
# The control of car in the environment makes by only one number, angle of turning
def command(lane_output, angle_output, lidar_output):
    
    global lidar_target_names
    global lane_target_names
    global counter
    global obstacleDetected
    
    action = '0'
    
    
    if counter == 15:
        counter = 0
        obstacleDetected = False
    
    # This condition is uses to give the car time to go over the next lane 
    if ((obstacleDetected) & (lidar_target_names[lidar_output] != 'Obstacle')):
        counter += 1
        return '0'
    
    # The main priority gives to lidar model, rather than camera model
    # Depending on what object is detected, system will make decision how to control the car
    if lidar_target_names[lidar_output] == 'Human':
        action = '-999'     # Stop moving
    elif lidar_target_names[lidar_output] == 'Obstacle':
        obstacleDetected = True
        if lane_target_names[lane_output] == 'Left Lane':
            action = '45'   # move to right lane
        else:
            action = '-45'  # move to left lane
    else:
        action = str(angle_output)
    
    return action

In [7]:
environment.listen()
print(environment.connection)

action = '0#_#_'

while(True):
    
    # Send the action and get next images to predict
    cameraImage, lidarImage = environment.step(action)
    
    # Make predictions
    camera_output = camera_model.predict(cameraImage)
    lidar_output = lidar_model.predict(lidarImage)
    
    # Calculate turning angle
    angle = camera_output[1][0][0] * 60
    
    # Form a new command
    action = command(np.argmax(camera_output[0]), angle, np.argmax(lidar_output)) + '#' + lane_target_names[np.argmax(camera_output[0])] + '#' + lidar_target_names[np.argmax(lidar_output)]
    
    print(action)

<socket.socket fd=2292, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7777), raddr=('127.0.0.1', 52063)>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
2.3091655#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1.4691818#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
3.1437986#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
4.7462974#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2.2967384#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1.5338205#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2.9774055#Right Lane#Clear path
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
5.387613#Right Lane

ConnectionAbortedError: [WinError 10053] Программа на вашем хост-компьютере разорвала установленное подключение